<a href="https://colab.research.google.com/github/syedsajidhussain/ML-Assessment/blob/main/ML_Engineer_Take_Home_Assessment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [90]:
! pip install bs4 lxml kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [101]:
!ls

'=1.0.0'
'10th qualification.zip'
'12 marks sheet.zip'
'12th qualification.zip'
'1533011601895 (1).jpg'
 1533011601895.jpg
'15366916999311891873372 (1).jpg'
 15366916999311891873372.jpg
'15366918810141436194623 (1).jpg'
 15366918810141436194623.jpg
 1-s2.0-S1053811920310521-main.pdf
'2022-03-23 12-15.pdf'
 26524---619.RPT
 Aadhaar.pdf
 a-comprehensive-tutorial-on-deep-learning-part-1.pdf
 A-Computer-Aided-Diagnosis-of-Brain-Tumors-Using-a-Fine-Tuned-YOLO-based-Model-with-Transfer-Learning.pdf
'advance in IRB till survey.docx'
 advjavamajor.docx
 agreement.pdf
'all documents.pdf'
'A novel method for Glioma tumor detection , segmentation and Classification using deeplearning.gdoc'
'Appointment  letter  for Syed Sajid Hussain - MAILab - Dec 3rd 2021 (1).pdf'
'Appointment_letter_Sajid_2023 (1).pdf'
'BeautyPlus_20180812144631989_save (1).jpg'
 BeautyPlus_20180812144631989_save.jpg
 Brain_Tumor_Classification.ipynb
'btech provisional.zip'
 Busse-August2020_Article_HowToWriteAndPublishAResear

In [97]:
import os
os.environ['KAGGLE_USERNAME'] = 'bilalyousaf0014'
os.environ['KAGGLE_KEY'] = '11031bc21c5e3ec23585dbe17dc4267d'

In [102]:
!kaggle datasets download -d bilalyousaf0014/ml-engineer-assessment-dataset

100% 78.6M/78.6M [00:04<00:00, 24.9MB/s]
100% 78.6M/78.6M [00:04<00:00, 18.9MB/s]


In [103]:
! unzip /content/drive/MyDrive/ml-engineer-assessment-dataset.zip

Archive:  /content/drive/MyDrive/ml-engineer-assessment-dataset.zip
  inflating: assessment_dataset/images/00001.jpeg  
  inflating: assessment_dataset/images/00008.jpeg  
  inflating: assessment_dataset/images/00017.jpeg  
  inflating: assessment_dataset/images/00022.jpeg  
  inflating: assessment_dataset/images/00048.jpeg  
  inflating: assessment_dataset/images/00055.jpeg  
  inflating: assessment_dataset/images/001.jpeg  
  inflating: assessment_dataset/images/1.jpeg  
  inflating: assessment_dataset/images/1001524.jpeg  
  inflating: assessment_dataset/images/1005343.jpeg  
  inflating: assessment_dataset/images/1049854.jpeg  
  inflating: assessment_dataset/images/1072860.jpeg  
  inflating: assessment_dataset/images/1120419.jpeg  
  inflating: assessment_dataset/images/1146885.jpeg  
  inflating: assessment_dataset/images/2.jpeg  
  inflating: assessment_dataset/images/3.jpeg  
  inflating: assessment_dataset/images/4.jpeg  
  inflating: assessment_dataset/images/5.jpeg  
  infl

In [104]:
import os
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from torchvision.models import resnet18
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor


In [105]:
class Model(nn.Module):
    def __init__(self, num_species):
        super(Model, self).__init__()
        self.backbone = resnet18(pretrained=True)
        self.have_object = nn.Linear(512, 1)
        self.bbox = nn.Linear(512, 4)
        self.cat_or_dog = nn.Linear(512, 2)
        self.specie = nn.Linear(512, num_species)
    
    def forward(self, input):
        features = self.backbone(input)
        have_object = torch.sigmoid(self.have_object(features))
        bbox = self.bbox(features)
        cat_or_dog = torch.sigmoid(self.cat_or_dog(features))
        specie = torch.sigmoid(self.specie(features))
        
        return {"bbox": bbox, "object": have_object, "cat_or_dog": cat_or_dog, "specie": specie}


In [106]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, dataset_path, train_list, train=True):
        self.images = []
        self.labels = []

        if train:
            file_list = train_list
        else:
            file_list = val_list
        
        for file in file_list:
            image_path = os.path.join(dataset_path, "images", file)
            self.images.append(image_path)

            label_path = os.path.join(dataset_path, "labels", file)
            label = np.load(label_path, allow_pickle=True)
            self.labels.append(label)
        
        # Implement any preprocessing and augmentations you need
        
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, index):
        image_path = self.images[index]
        image = self.preprocess(image_path)  # Implement the preprocessing function
        label = self.labels[index]
        return image, label



In [107]:
def train(epochs, train_list, val_list, model_weights):
    # Define constants
    num_species = 9
    batch_size = 16
    learning_rate = 0.001
    
    # Create the model
    model = Model(num_species)
    
    # Define the loss functions
    criterion_bbox = nn.MSELoss()
    criterion_object = nn.BCELoss()
    criterion_cat_or_dog = nn.CrossEntropyLoss()
    criterion_specie = nn.CrossEntropyLoss()
    
    # Create the optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
    # Create the training dataset and data loader
    training_dataset = CustomDataset("/content/drive/MyDrive/assessment_dataset", train_list, "labels")
    training_loader = DataLoader(training_dataset, batch_size=batch_size, shuffle=True)
    
    # Create the validation dataset and data loader
    validation_dataset = CustomDataset("/content/drive/MyDrive/assessment_dataset", val_list, "labels")
    validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)
    
    # Move the model to the GPU if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    # Training loop
    for epoch in range(epochs):
        model.train()
        epoch_loss = 0.0
        
        for inputs, labels in training_loader:
            inputs = inputs.to(device)
            labels = {
                "bbox": labels["bbox"].to(device),
                "object": labels["object"].to(device),
                "cat_or_dog": labels["cat_or_dog"].to(device),
                "specie": labels["specie"].to(device)
            }
            
            optimizer.zero_grad()
            outputs = model(inputs)
            
            loss_bbox = criterion_bbox(outputs["bbox"], labels["bbox"])
            loss_object = criterion_object(outputs["object"], labels["object"])
            loss_cat_or_dog = criterion_cat_or_dog(outputs["cat_or_dog"], labels["cat_or_dog"])
            loss_specie = criterion_specie(outputs["specie"], labels["specie"])
            
            loss = loss_bbox + loss_object + loss_cat_or_dog + loss_specie
            loss.backward()
            optimizer.step()
            
            epoch_loss += loss.item() * inputs.size(0)
        
        epoch_loss /= len(training_dataset)
        print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}")
    
    # Save the trained model
    torch.save(model.state_dict(), model_weights)
    print("Trained model saved successfully!")

def test(model_weights, test_list):
    # Load the trained model
    model = Model(num_species)
    model.load_state_dict(torch.load(model_weights))
    
    # Create the test dataset and data loader
    test_dataset = CustomDataset("/content/drive/MyDrive/assessment_dataset", test_list, "labels")
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
    
    # Move the model to the GPU if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    # Set the model to evaluation mode
    model.eval()
    
    # Iterate over the test dataset
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        outputs = model(inputs)
        
        # Process the model outputs and perform any required post-processing
        
        # Print or save the predictions for further analysis or visualization
        print(outputs)
        
def visualize_results():
    # Implement the code to visualize the training and testing results
    pass


In [110]:
train_list = np.load("/content/drive/MyDrive/assessment_dataset/train_list.npy", allow_pickle=True).tolist()
val_list = np.load("/content/drive/MyDrive/assessment_dataset/val_list.npy", allow_pickle=True).tolist()

model_weights = "model.pth"
train(50, train_list, val_list, model_weights)


FileNotFoundError: ignored

In [109]:
test_list = np.load("test_list.npy")
model_weights = "model.pth"
test(model_weights, test_list)


FileNotFoundError: ignored

In [83]:
visualize_results()
